In [1]:
import heapq
import math
import os
import re
import time
from typing import List, Optional, Tuple

import aocd
import numpy as np
from IPython.display import HTML, clear_output, display
from scipy.ndimage import convolve

In [2]:
p = aocd.get_puzzle(year=2024, day=18)

In [3]:
def process_data(data, shape, c):
    
    grid = np.full(shape, ".", dtype=str)
    for i, coord in enumerate(data.split("\n")):
   
        if i >= c:
            print("Stopped processing at", last_coord, "blocks")
            break
        row, col = map(int, coord.split(","))
        grid[col, row] = "#"
        last_coord = coord
    
    return grid

In [4]:
def get_data(test_data: bool = False, num_blocks = None):
    if test_data:
        data = p.examples[0].input_data
        if not num_blocks:
            num_blocks = 20
        grid = process_data(data, (7,7), num_blocks)
    else:
        data = p.input_data
        if not num_blocks:
            num_blocks = 1024
        grid = process_data(data, (71,71), num_blocks)
    grid[0, 0] = "S"
    grid[grid.shape[0]-1, grid.shape[1]-1] = "E"
    return grid

In [5]:
get_data(test_data=True, num_blocks=12)

Stopped processing at 5,1 blocks


array([['S', '.', '.', '#', '.', '.', '.'],
       ['.', '.', '#', '.', '.', '#', '.'],
       ['.', '.', '.', '.', '#', '.', '.'],
       ['.', '.', '.', '#', '.', '.', '#'],
       ['.', '.', '#', '.', '.', '#', '.'],
       ['.', '#', '.', '.', '#', '.', '.'],
       ['#', '.', '#', '.', '.', '.', 'E']], dtype='<U1')

In [6]:
Coord = Tuple[int, int]

class MazePathFinder:
    """
    Find paths minimizing:
        cost = steps + turn_cost * turns

    grid: 2D numpy array of characters "#", ".", "S", "E"
    """

    DIRS = [
        (-1, 0, "U"),
        (1, 0, "D"),
        (0, -1, "L"),
        (0, 1, "R"),
    ]

    def __init__(self, grid: np.ndarray, turn_cost: int = 1000):
        self.grid = grid
        self.turn_cost = float(turn_cost)

        self.rows, self.cols = grid.shape

        (sr, sc), = np.argwhere(grid == "S")
        (er, ec), = np.argwhere(grid == "E")

        self.start: Coord = (int(sr), int(sc))
        self.end: Coord = (int(er), int(ec))

        self.walkable = (grid != "#")

        self._dist = None
        self._parents = None
        self._best_cost = None
        self._end_states = None

    # ---------- PUBLIC API ----------

    def find_optimal_paths(self, max_paths: Optional[int] = None):
        if self._dist is None:
            self._run_dijkstra()

        if not self._end_states:
            print("⚠️ No valid path from S to E exists in this maze.")
            return []

        return self._backtrack_all_paths(max_paths=max_paths)

    def best_cost(self) -> Optional[float]:
        if self._dist is None:
            self._run_dijkstra()
        if self._best_cost is None:
            print("⚠️ No path found — best cost is undefined.")
        return self._best_cost

    @staticmethod
    def path_to_directions(path: List[Coord]) -> List[str]:
        dirs = []
        for (r1, c1), (r2, c2) in zip(path, path[1:]):
            dr, dc = r2 - r1, c2 - c1
            if dr == 1 and dc == 0: dirs.append("D")
            elif dr == -1 and dc == 0: dirs.append("U")
            elif dr == 0 and dc == 1: dirs.append("R")
            elif dr == 0 and dc == -1: dirs.append("L")
        return dirs

    def count_steps_and_turns(self, path: List[Coord]):
        dirs = self.path_to_directions(path)
        steps = len(dirs)
        turns = sum(1 for a, b in zip(dirs, dirs[1:]) if a != b)
        return steps, turns, dirs

    def path_cost(self, path: List[Coord]) -> float:
        steps, turns, _ = self.count_steps_and_turns(path)
        return steps + self.turn_cost * turns

    # ---------- INTERNALS ----------

    def _run_dijkstra(self):
        rows, cols = self.rows, self.cols
        walkable = self.walkable
        DIRS = self.DIRS
        DIR_COUNT = len(DIRS)
        TURN_COST = self.turn_cost
        (start_r, start_c) = self.start
        (end_r, end_c) = self.end

        dist = np.full((rows, cols, DIR_COUNT), math.inf, dtype=float)
        parents = np.empty((rows, cols, DIR_COUNT), dtype=object)
        parents[:] = None

        pq = []

        # Initial expansion from S
        for d_idx, (dr, dc, _) in enumerate(DIRS):
            nr, nc = start_r + dr, start_c + dc
            if 0 <= nr < rows and 0 <= nc < cols and walkable[nr, nc]:
                dist[nr, nc, d_idx] = 1.0
                parents[nr, nc, d_idx] = [(start_r, start_c, None)]
                heapq.heappush(pq, (1.0, nr, nc, d_idx))

        # Dijkstra
        while pq:
            cost, r, c, d_idx = heapq.heappop(pq)
            if cost != dist[r, c, d_idx]:
                continue

            for nd_idx, (dr, dc, _) in enumerate(DIRS):
                nr, nc = r + dr, c + dc
                if not (0 <= nr < rows and 0 <= nc < cols):
                    continue
                if not walkable[nr, nc]:
                    continue

                step_cost = 1.0
                turn_penalty = 0.0 if nd_idx == d_idx else TURN_COST
                new_cost = cost + step_cost + turn_penalty

                old_cost = dist[nr, nc, nd_idx]

                if new_cost < old_cost:
                    dist[nr, nc, nd_idx] = new_cost
                    parents[nr, nc, nd_idx] = [(r, c, d_idx)]
                    heapq.heappush(pq, (new_cost, nr, nc, nd_idx))

                elif new_cost == old_cost:
                    if parents[nr, nc, nd_idx] is None:
                        parents[nr, nc, nd_idx] = []
                    parents[nr, nc, nd_idx].append((r, c, d_idx))

        # Determine best end-state
        best_cost = math.inf
        end_states = []

        for d_idx in range(DIR_COUNT):
            cst = dist[end_r, end_c, d_idx]
            if cst < best_cost:
                best_cost = cst
                end_states = [(end_r, end_c, d_idx)]
            elif cst == best_cost:
                end_states.append((end_r, end_c, d_idx))

        if math.isinf(best_cost):
            self._best_cost = None
            self._end_states = []
            print("⚠️ No path exists from S to E in this maze.")
            return

        self._dist = dist
        self._parents = parents
        self._best_cost = best_cost
        self._end_states = end_states

    def _backtrack_all_paths(self, max_paths):
        if not self._end_states:
            return []

        parents = self._parents
        start = self.start
        all_paths = []

        stack = []
        for (er, ec, d_idx) in self._end_states:
            stack.append((er, ec, d_idx, [(er, ec)]))

        while stack:
            r, c, d_idx, path_back = stack.pop()
            parent_list = parents[r, c, d_idx]
            if parent_list is None:
                continue

            for pr, pc, pdir in parent_list:
                new_path_back = path_back + [(pr, pc)]

                if pdir is None:
                    if (pr, pc) != start:
                        continue
                    full_path = list(reversed(new_path_back))
                    all_paths.append(full_path)

                    if max_paths and len(all_paths) >= max_paths:
                        stack = []
                        break
                else:
                    stack.append((pr, pc, pdir, new_path_back))

        # Deduplicate
        unique = []
        seen = set()
        for p in all_paths:
            t = tuple(p)
            if t not in seen:
                seen.add(t)
                unique.append(p)

        return unique

## Part 1 - Dijkstra

In [7]:
%%time
grid = get_data(test_data=False, num_blocks=1024)
finder = MazePathFinder(grid, turn_cost=0)

paths = finder.find_optimal_paths(max_paths=50)


if paths:
    min_cost = finder.best_cost()
    print(len(paths), "optimal paths found with ", int(min_cost), " steps")

Stopped processing at 67,32 blocks
50 optimal paths found with  268  steps
CPU times: user 28.5 ms, sys: 1.17 ms, total: 29.6 ms
Wall time: 29.1 ms


## Part 2 

In [8]:
%%time
for i in range(2932, 3000):
    print("Testing with", i, "blocks")
    grid = get_data(test_data=False, num_blocks=i)
    finder = MazePathFinder(grid, turn_cost=0)
    
    paths = finder.find_optimal_paths(max_paths=50)
    
    
    if not paths:
        break

Testing with 2932 blocks
Stopped processing at 50,20 blocks
Testing with 2933 blocks
Stopped processing at 63,2 blocks
Testing with 2934 blocks
Stopped processing at 64,11 blocks
⚠️ No path exists from S to E in this maze.
⚠️ No valid path from S to E exists in this maze.
CPU times: user 10.2 ms, sys: 1.22 ms, total: 11.5 ms
Wall time: 10.8 ms


## Illustration

In [9]:
def html_grid(grid):
    colors = {
        "#": ("█", "#8d6e63"),  # Warm brown walls
        ".": ("·", "#4a4a4a"),  # Subtle grey dots
        "O": ("▣", "#f44336"),
        "@": ("◉", "#4caf50"),
    }
    background = "#2d2d2d"

    cells = ""
    for row in grid:
        for c in row:
            char, color = colors.get(c, (c, "#333333"))
            cells += f'<span style="color:{color};">{char}</span>'
        cells += "<br>"

    html = f"""
    <div style="font-family:monospace;font-size:16px;line-height:1.1;background:{background};padding:15px;border-radius:10px;display:inline-block;">
    {cells}
    </div>
    """
    return html

In [10]:
grid = get_data(test_data=False, num_blocks=2933)
finder = MazePathFinder(grid, turn_cost=0)
paths = finder.find_optimal_paths(max_paths=50)

for r, c in paths[0]:
    grid[r, c] = "@"


clear_output(wait=True)
display(HTML(html_grid(grid)))